# Building Multi-Agents - Solving a Complex Task with a Multi-Agent Hierarchy

This notebook is part of the [Hugging Face Agents Course unit 2](https://huggingface.co/agents-course/notebooks/blob/main/unit2/smolagents/code_agents.ipynb), a free Course from beginner to expert, where you learn to build Agents.


## Description
In this notebook, we examine the use of mutliple agents.

## The Problem
The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

```
👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

```
Let's build this!


## Load Imports

In [19]:
import math
import os
from typing import Optional, Tuple

from dotenv import load_dotenv
from huggingface_hub import notebook_login
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool, tool




##### Login to the Hugging Face Hub to Have Access to the Serveless Inference API

Note: You will need to add your token when prompted.


`HF_TOKEN_INFERENCE2`

In [20]:
# Load environment variables from .env
load_dotenv()

True

In [21]:
# Get Hugging Face Token
HF_TOKEN_INFERENCE2 = os.environ.get("HF_TOKEN_INFERENCE2")

# Get SerApi API Key
SERPAPI_API_KEY = os.environ.get("SERPAPI_API_KEY")

In [5]:
notebook_login()

### Make a Tool to Get the Cargo Plane Transfer Time

In [6]:
@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use `Together AI`, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [10]:
model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [11]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [12]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [13]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

4. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

5. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOorMoHVdO0wmuvvgMET7YuYOB5Hx8mWZ0NrDF_Q
2CFk41HVkr-H2)
Date published: Apr 20, 2023
Source: LA Today

Los Angeles, CA. The legendary Batcave is found in Bronson Canyon, at the Bronson Caves. This is a section of the 
expansive Griffith Park in Los ...

7. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow ... The Batman filming location: James R 
Thompson Center, West Randolph Street, Chicago.

Out: None

[Step 1: Duration 6.89 seconds| Input tokens: 2,237 | Output tokens: 152]

...

[Step 8: Duration 4.77 seconds| Input tokens: 88,406 | Output tokens: 1,625]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Create DataFrame for Batman filming locations                                                                  
  batman_df = pd.DataFrame(list(travel_times.items()), columns=['Location', 'Travel Time to Gotham (hours)'])      
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame(list(supercar_travel_times.items()), columns=['Location', 'Travel Time to Gotham      
  (hours)'])                                                                                                       
                                                                                                                   
  # Combine both DataFrames                                                                                        
  combined_df = pd.concat([batman_df, supercar_df], keys=['Batman Filming Locations', 'Supercar Factories'])       
                                                                                                                   
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                                                                      Location  \
Batman Filming Locations 0         Necropolis Cemetery, Glasgow, Scotland, UK   
                         1           St George's Hall, Liverpool, England, UK   
                         2     Royal Liver Building, Pier Head, Liverpool, UK   
                         3  Knebworth House, Knebworth, Hertfordshire, Eng...   
                         4  Acton Lane Power Station, Acton Lane, Acton, L...   
                         5  James R Thompson Center, West Randolph Street,...   
                         6  Bronson Caves, Bronson Canyon, Los Angeles, CA...   
Supercar Factories       0                           Ferrari Maranello, Italy   
                         1            Lamborghini Sant'Agata Bolognese, Italy   
                         2               Pagani San Cesario sul Panaro, Italy   
                         3                                 McLaren Woking, UK   
                         4                     Koenigsegg Angelholmen, Sweden   

                            Travel Time to Gotham (hours)  
Batman Filming Locations 0                           8.60  
                         1                           8.81  
                         2                           8.81  
                         3                           9.15  
                         4                           9.16  
                         5                           2.68  
                         6                           6.78  
Supercar Factories       0                          10.72  
                         1                          10.74  
                         2                          10.74  
                         3                           9.14  
                         4                          10.04  

[Step 9: Duration 8.71 seconds| Input tokens: 108,410 | Output tokens: 1,783]

In [14]:
result

Location  \
Batman Filming Locations 0         Necropolis Cemetery, Glasgow, Scotland, UK   
                         1           St George's Hall, Liverpool, England, UK   
                         2     Royal Liver Building, Pier Head, Liverpool, UK   
                         3  Knebworth House, Knebworth, Hertfordshire, Eng...   
                         4  Acton Lane Power Station, Acton Lane, Acton, L...   
                         5  James R Thompson Center, West Randolph Street,...   
                         6  Bronson Caves, Bronson Canyon, Los Angeles, CA...   
Supercar Factories       0                           Ferrari Maranello, Italy   
                         1            Lamborghini Sant'Agata Bolognese, Italy   
                         2               Pagani San Cesario sul Panaro, Italy   
                         3                                 McLaren Woking, UK   
                         4                     Koenigsegg Angelholmen, Sweden   

                            Travel Time to Gotham (hours)  
Batman Filming Locations 0                           8.60  
                         1                           8.81  
                         2                           8.81  
                         3                           9.15  
                         4                           9.16  
                         5                           2.68  
                         6                           6.78  
Supercar Factories       0                          10.72  
                         1                          10.74  
                         2                          10.74  
                         3                           9.14  
                         4                          10.04

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [15]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find Batman filming locations worldwide.
- We need to calculate the travel time for a cargo plane to Gotham for each filming location.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- List of Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations" and verify from reliable sources like IMDb, official movie 
websites, or reputable news articles.
- Coordinates of each Batman filming location.
  - Source: Geocoding services or verified maps and location databases.
- List of supercar factories worldwide.
  - Source: Web search for "supercar factories" and verify from reliable sources like automotive news websites or 
official manufacturer websites.
- Coordinates of each supercar factory.
  - Source: Geocoding services or verified maps and location databases.
- Cruising speed of a typical cargo plane.
  - Source: Aviation industry standards or reliable aviation websites.

### 1.3. Facts to derive
- Travel time for a cargo plane from each Batman filming location to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each filming location and 
Gotham.
- Travel time for a cargo plane from each supercar factory to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each supercar factory and 
Gotham.

## 2. Plan
1. Perform a web search for "Batman filming locations" and compile a list of locations.
2. For each Batman filming location, perform a web search to find its coordinates.
3. Perform a web search for "supercar factories" and compile a list of factories.
4. For each supercar factory, perform a web search to find its coordinates.
5. Perform a web search to find the cruising speed of a typical cargo plane.
6. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
7. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
8. Create a pandas dataframe with Batman filming locations and their travel times to Gotham.
9. Create a pandas dataframe with supercar factories and their travel times to Gotham.
10. Finalize the dataframes and prepare them for the final answer.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-686338ce-4ffaf0d46c7c43d65d7d06f5;75ebf7d7-532e-4381-9f05-e59e4bcf50a4)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.05 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-686338ce-4ffaf0d46c7c43d65d7d06f5;75ebf7d7-532e-4381-9f05-e59e4bcf50a4)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

The Detailed Report


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs.

➡️ We need to improve the structure of our system.

## Splitting the task between two agents
Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:

Each agent is more focused on its core task, thus more performant
Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.
Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [22]:
model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider=SERPAPI_API_KEY),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

ValueError: Missing API key. Make sure you have 'SERPER_API_KEY' in your env variables.

The manager agent will need to do some mental heavy lifting.

You will need to get an OPENAI Key.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a planning_interval to the mix.

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "/Users/lancehester/Documents/ai_agent_hugging_face_course/imgs/saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
# This will generate something like this, helping us understand the structure and relationship between agents and tools used:

manager_agent.visualize()

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in 1. Search for Batman filming locations, then 2. Find supercar factories, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]